In [1]:
import findspark 
findspark.init()
findspark.find()


'C:\\apachespark\\spark-2.4.7-bin-hadoop2.7'

In [2]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName('testapp').getOrCreate()

In [4]:
d1= spark.read.csv('txnsSmall2')

In [5]:
d1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)



In [6]:
d1.show(5)

+--------+----------+-------+------+------------------+--------------------+-----------+----------+-------+
|     _c0|       _c1|    _c2|   _c3|               _c4|                 _c5|        _c6|       _c7|    _c8|
+--------+----------+-------+------+------------------+--------------------+-----------+----------+-------+
|   txnid|   txndate| custid|amount|          category|         subcategory|       city|     state|txntype|
|00000000|06-26-2011|4007024|040.33|Exercise & Fitness|Cardio Machine Ac...|Clarksville| Tennessee| credit|
|00000001|05-26-2011|4006742|198.44|Exercise & Fitness|Weightlifting Gloves| Long Beach|California| credit|
|00000002|06-01-2011|4009775|005.58|Exercise & Fitness|Weightlifting Mac...|    Anaheim|California| credit|
|00000003|06-05-2011|4002199|198.19|        Gymnastics|    Gymnastics Rings|  Milwaukee| Wisconsin| credit|
+--------+----------+-------+------+------------------+--------------------+-----------+----------+-------+
only showing top 5 rows



In [7]:
d1= spark.read.option('header',True).option('inferSchema',True).csv('txnsSmall2')

In [8]:
d1.show(5)

+-----+----------+-------+------+------------------+--------------------+-----------+----------+-------+
|txnid|   txndate| custid|amount|          category|         subcategory|       city|     state|txntype|
+-----+----------+-------+------+------------------+--------------------+-----------+----------+-------+
|    0|06-26-2011|4007024| 40.33|Exercise & Fitness|Cardio Machine Ac...|Clarksville| Tennessee| credit|
|    1|05-26-2011|4006742|198.44|Exercise & Fitness|Weightlifting Gloves| Long Beach|California| credit|
|    2|06-01-2011|4009775|  5.58|Exercise & Fitness|Weightlifting Mac...|    Anaheim|California| credit|
|    3|06-05-2011|4002199|198.19|        Gymnastics|    Gymnastics Rings|  Milwaukee| Wisconsin| credit|
|    4|12-17-2011|4002613| 98.81|       Team Sports|        Field Hockey|Nashville  | Tennessee| credit|
+-----+----------+-------+------+------------------+--------------------+-----------+----------+-------+
only showing top 5 rows



In [9]:
from pyspark.sql.types import *


In [16]:
d2 = StructType([StructField('txnid',IntegerType(),True),StructField('txndate',StringType(),True),
                StructField('custid',IntegerType(),True),StructField('amount',DoubleType(),True),
                StructField('category',StringType(),True),StructField('subcategory',StringType(),True),
                StructField('city',StringType(),True),StructField('state',StringType(),True),
                StructField('txntype',StringType(),True)])

In [18]:
dataf=spark.read.schema(d2).csv('txnsSmall')

In [19]:
dataf.show(5)

+-----+----------+-------+------+------------------+--------------------+-----------+----------+-------+
|txnid|   txndate| custid|amount|          category|         subcategory|       city|     state|txntype|
+-----+----------+-------+------+------------------+--------------------+-----------+----------+-------+
|    0|06-26-2011|4007024| 40.33|Exercise & Fitness|Cardio Machine Ac...|Clarksville| Tennessee| credit|
|    1|05-26-2011|4006742|198.44|Exercise & Fitness|Weightlifting Gloves| Long Beach|California| credit|
|    2|06-01-2011|4009775|  5.58|Exercise & Fitness|Weightlifting Mac...|    Anaheim|California| credit|
|    3|06-05-2011|4002199|198.19|        Gymnastics|    Gymnastics Rings|  Milwaukee| Wisconsin| credit|
|    4|12-17-2011|4002613| 98.81|       Team Sports|        Field Hockey|Nashville  | Tennessee| credit|
+-----+----------+-------+------+------------------+--------------------+-----------+----------+-------+
only showing top 5 rows



In [20]:
dataf.write.parquet('dataf.parquet')

In [21]:
datafp = spark.read.parquet('dataf.parquet')

In [22]:
datafp.show(5)

+-----+----------+-------+------+------------------+--------------------+-----------+----------+-------+
|txnid|   txndate| custid|amount|          category|         subcategory|       city|     state|txntype|
+-----+----------+-------+------+------------------+--------------------+-----------+----------+-------+
|    0|06-26-2011|4007024| 40.33|Exercise & Fitness|Cardio Machine Ac...|Clarksville| Tennessee| credit|
|    1|05-26-2011|4006742|198.44|Exercise & Fitness|Weightlifting Gloves| Long Beach|California| credit|
|    2|06-01-2011|4009775|  5.58|Exercise & Fitness|Weightlifting Mac...|    Anaheim|California| credit|
|    3|06-05-2011|4002199|198.19|        Gymnastics|    Gymnastics Rings|  Milwaukee| Wisconsin| credit|
|    4|12-17-2011|4002613| 98.81|       Team Sports|        Field Hockey|Nashville  | Tennessee| credit|
+-----+----------+-------+------+------------------+--------------------+-----------+----------+-------+
only showing top 5 rows



SQL Approach 
        #Find the total number of transaction done by cash
		# Find the Highest Selling Category
		# Find the lowest selling Category
		# Find the lowest selling subcategory in highest selling category

In [24]:
datafp.registerTempTable('datafp')

In [26]:
spark.sql("select count(*) as counttr from datafp where txntype = 'cash'").show()

+-------+
|counttr|
+-------+
|     64|
+-------+



In [28]:
spark.sql("select category, count(*) as countH from datafp group by category order by countH desc limit 1").show()

+------------------+------+
|          category|countH|
+------------------+------+
|Outdoor Recreation|    57|
+------------------+------+



In [29]:
spark.sql("select category, count(*) as countH from datafp group by category order by countH limit 1").show()

+--------+------+
|category|countH|
+--------+------+
| Puzzles|     7|
+--------+------+



In [31]:
spark.sql("select subcategory,count(*) as countS from datafp where category = (select category from datafp group by category order by count(txnid) desc limit 1 ) group by subcategory order by count(*) limit 1").show() 

+-----------+------+
|subcategory|countS|
+-----------+------+
|       Golf|     1|
+-----------+------+



#programming approch 
        #Find the total number of transaction done by cash
		# Find the Highest Selling Category
		# Find the lowest selling Category
		# Find the lowest selling subcategory in highest selling category

In [51]:
from pyspark.sql.functions import *

In [52]:
datafp.where("txntype ='cash'").count()

64

In [55]:
datafp.groupBy('category').agg({'txnid':'count'}).withColumnRenamed("count(txnid)", "countt").sort(desc("countt")).show(1)

+------------------+------+
|          category|countt|
+------------------+------+
|Outdoor Recreation|    57|
+------------------+------+
only showing top 1 row



In [56]:
datafp.groupBy('category').agg({'txnid':'count'}).withColumnRenamed("count(txnid)", "countt").sort(asc("countt")).show(1)

+--------+------+
|category|countt|
+--------+------+
| Puzzles|     7|
+--------+------+
only showing top 1 row



In [50]:
datafp.groupBy('subcategory').agg({'txnid':'count'}).sort(Asc(count('txnid')))

NameError: name 'Asc' is not defined